# 5.5 텐서플로우를 이용한 신경망 만들기 예제

이번에는 텐서플로우를 이용해 신경망을 직접 만들어 보도록 하겠습니다. 

여기에서는 간단한 신경망을 구축하고, 이를 MNIST라는 손글씨 데이터에 대해 학습시킨 후 평가함으로써 얼마만큼 정확하게 맞추는 지 확인해 보도록 하겠습니다.

POINT
* MNIST 데이터에 대해 이해하고 활용합니다.
* 신경망 모델을 만듭니다.
* 신경망을 MNIST 데이터에 대해 학습시킵니다.
* 학습된 신경망의 성능을 평가합니다.

### 5.5.1. 필요한 모듈 불러오기

텐서플로우 라이브러리와 numpy 라이브러리를 불러 옵니다.

In [1]:
import tensorflow as tf
import numpy as np

MNIST란 손으로 쓴 0~9의 숫자 모음 데이터입니다. 
* MNIST는 학습용 데이터 55,000개, 검증용 데이터 5,000개, 테스트용 데이터 10,000개로 구성되어 있습니다.
* MNIST 데이터 각각은 이미지와 레이블의 쌍으로 구성됩니다.
* 텐서플로우에는 편의를 위해 mnist가 예제로 포함되어 있습니다. 
* 텐서플로우에서는 mnist 데이터를 다루기 편리하도록 몇가지 도구를 제공하므로 우리는 이를 활용하도록 하겠습니다. 

### 5.5.2. MNIST 데이터 다운로드하기

tensorflow.examples.tutorials.mnist 로부터 input_data를 불러온 다음에, 이를 이용하여 mnist 데이터를 다운로드합니다.
* 이때 one_hot을 True로 설정해 줍니다. 
* one_hot 방식이란, 적덜한 개수의 이진수 (0 또는 1) 로 표현해주는 것을 의미합니다.
* MNIST에서는 one_hot 방식으로 3은 (0,0,0,1,0,0,0,0,0,0)로, 7은 (0,0,0,0,0,0,0,1,0,0)로 표현됩니다. 

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/mnist/", one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


### 5.5.3. MNIST 데이터 살펴보기

len()을 이용해 학습용 데이터의 이미지의 개수와 레이블의 개수를 확인해 봅니다.

In [3]:
print len(mnist.train.labels)
print len(mnist.train.images)

55000
55000


위에서 설명한대로 이미지와 레이블 각각 55,000개임을 확인하였습니다. 

이번에는 검증용 데이터와 테스트용 데이터의 개수를 확인해 봅니다. num_exmples를 활용하면 간단히 개수를 확인할 수 있습니다.

In [4]:
print mnist.validation.num_examples
print mnist.test.num_examples

5000
10000


역시 위에서 설명한대로 검증용 데이터는 5,000개, 테스트용 데이터는 10,000개인 것을 확인할 수 있었습니다. 

shape를 활용하여 테스트용 데이터 중 레이블의 형태를 확인할 수 있습니다. 

In [5]:
mnist.test.labels.shape

(10000, 10)

(10000, 10) 의 형태를 가지는 것을 확인할 수 있습니다. 테스트용 데이터는 총 10,000개이며 각 데이터는 one_hot으로 되어 있기 때문에 10개가 되는 것입니다. 

학습용 데이터 중 레이블의 내용은 아래와 같이 확인 가능합니다.

In [6]:
mnist.test.labels[0:5, :]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

[] 안의 0:5 의 의미는 0부터 4행까지 총 5개의 행을 보겠다는 의미이며, 뒤쪽 : 의 의미는 모든 열을 보겠다는 의미가 됩니다. 따라서 (5, 10) 의 형태로 내용이 출력된 것입니다. 각 행에서 1의 위치에 주목하면 10진수로 어떤 값을 의미하는 지 파악이 가능합니다. 

이를 10진수로 변환하면 아래와 같습니다.

In [7]:
mnist.test.cls = np.argmax(mnist.test.labels, axis=1)
mnist.test.cls[0:5]

array([7, 2, 1, 0, 4])

np.argmax에서 axis=1로 설정함으로써 각 행에서 가장 큰 값을 가지는 인자의 값을 가져와 mnist.test.cls 에 할당하게 되는 것입니다. 

### 5.5.4. 하이퍼파라미터 설정해주기

학습률과 학습횟수를 설정합니다.

In [8]:
num_epochs = 30
learning_rate = 0.01

우리는 입력 계층 --> 은닉 계층 1 --> 은닉 계층 2 --> 출력 계층으로 구성된 신경망을 구축할 것입니다. 각 계층에서의 노드의 갯수는 아래와 같이 설정하도록 하겠습니다.
* 입력 계층의 노드 수 : 784 (MNIST 이미지 각각은 28x28의 해상도를 가지고 있기 때문에 입력 계층의 노드 수는 784가 됩니다.)
* 은닉 계층1의 노드 수 : 256 (은닉 계층에서의 노드 수는 일종의 하이퍼파라미터입니다. 여기에서는 적당하게 256개로 설정하였습니다.)
* 은닉 계층2의 노드 수 : 256 (은닉 계층에서의 노드 수는 일종의 하이퍼파라미터입니다. 여기에서는 적당하게 256개로 설정하였습니다.)
* 출력 계층의 노드 수 : 10 (0부터 9까지 총 10개의 클래스로 구성 (One_hot) 되기 때문에 출력 계층의 노드 수는 10이 됩니다.)

In [9]:
num_node_input = 28*28
num_node_hidden1 = 256
num_node_hidden2 = 256
num_node_output = 10

### 5.5.5. 모델 만들기

입력 계층을 플레이스홀더로 정의해 줍니다.

In [10]:
x_true = tf.placeholder(tf.float32, [None, num_node_input])
y_true = tf.placeholder(tf.float32, [None, num_node_output])

은닉 계층 1의 가중치와 편향을 설정해 줍니다.

In [11]:
weight_1 = tf.Variable(tf.truncated_normal([num_node_input, num_node_hidden1], stddev=0.01))
bias_1 = tf.Variable(tf.zeros([num_node_hidden1]))

은닉 계층 2의 가중치와 편향을 설정해 줍니다.

In [12]:
weight_2 = tf.Variable(tf.truncated_normal([num_node_hidden1, num_node_hidden2], stddev=0.01))
bias_2 = tf.Variable(tf.zeros([num_node_hidden2]))

출닉 계층의 가중치와 편향을 설정해 줍니다.

In [13]:
weight_3 = tf.Variable(tf.truncated_normal([num_node_hidden2, num_node_output], stddev=0.01))
bias_3 = tf.Variable(tf.zeros([num_node_output]))

은닉 계층 1, 은닉 계층 2, 출력 계층에서 연산을 수행하고 각각 relu를 이용해 non-linearity를 주게 됩니다.

In [14]:
hidden_1 = tf.nn.relu(tf.add(tf.matmul(x_true, weight_1), bias_1))
hidden_2 = tf.nn.relu(tf.add(tf.matmul(hidden_1, weight_2), bias_2))
y_pred = tf.nn.relu(tf.add(tf.matmul(hidden_2, weight_3), bias_3))

예측 값과 실제 값의 차를 이용해 cross entropy를 구해 주고 이를 평균한 값이 비용이 됩니다. 비용을 최소화하도록 학습시켜 주기 위해 optimizer를 정의해 줍니다. 

In [15]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### 5.5.6. 학습시키기

세션을 실행합니다.

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

MNIST의 training data는 55,000개나 되기 때문에 이를 batch로 묶어 주어 피드해 주게 됩니다.

In [17]:
batch_size = 100
total_batch = int(mnist.train.num_examples/batch_size)

batch_size 수 역시 하이퍼파라미터이기 때문에 적절히 설정해 주어야 합니다. total_batch는 학습용 데이터 전체 (55,000개) 를 batch_size 100으로 나누어주었기 때문에 550이 됩니다.

학습을 시작합니다. 
* 앞에서 정의한 num_epochs 만큼 학습을 진행합니다.
* batch로 100개씩 묶어서 피드하게 됩니다.
* 텐서플로우에서는 mnist 데이터의 피드를 편하게 하기 위해 next_batch()라는 함수를 제공합니다.

In [18]:
for epoch in range(num_epochs):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer, {x_true:batch_xs, y_true:batch_ys})
        total_cost += sess.run(cost, {x_true:batch_xs, y_true:batch_ys})
    
    print "Epoch : {%04d}" % (epoch + 1), "Cost : {:.3f}".format(total_cost / total_batch)
    
print "최적화가 완료되었습니다."

Epoch : {0001} Cost : 1.300
Epoch : {0002} Cost : 1.210
Epoch : {0003} Cost : 1.195
Epoch : {0004} Cost : 1.185
Epoch : {0005} Cost : 1.179
Epoch : {0006} Cost : 1.175
Epoch : {0007} Cost : 1.173
Epoch : {0008} Cost : 1.172
Epoch : {0009} Cost : 1.169
Epoch : {0010} Cost : 1.165
Epoch : {0011} Cost : 1.107
Epoch : {0012} Cost : 0.962
Epoch : {0013} Cost : 0.951
Epoch : {0014} Cost : 0.887
Epoch : {0015} Cost : 0.741
Epoch : {0016} Cost : 0.730
Epoch : {0017} Cost : 0.722
Epoch : {0018} Cost : 0.721
Epoch : {0019} Cost : 0.717
Epoch : {0020} Cost : 0.716
Epoch : {0021} Cost : 0.712
Epoch : {0022} Cost : 0.711
Epoch : {0023} Cost : 0.708
Epoch : {0024} Cost : 0.711
Epoch : {0025} Cost : 0.705
Epoch : {0026} Cost : 0.704
Epoch : {0027} Cost : 0.704
Epoch : {0028} Cost : 0.704
Epoch : {0029} Cost : 0.704
Epoch : {0030} Cost : 0.703
최적화가 완료되었습니다.


### 5.5.7. 평가하기

* 예측 값과 실제 값이 동일한 경우를 카운트해 줍니다. 
* 이를 실수로 캐스팅한 다음에 평균을 냄으로써 정확도를 정의합니다. 
* 세션을 수행하여 테스트 이미지와 레이블을 전달하여 정확도를 구할 수 있습니다. 

In [19]:
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print "정확도 :", sess.run(accuracy, {x_true: mnist.test.images,
                                     y_true: mnist.test.labels})

정확도 : 0.7791


지금까지 간단한 신경망을 구축하고 이를 MNIST 데이터에 학습시키고 평가해 보았습니다. 77.91%라는 정확도가 나쁘지는 않다고 생각될 지 모르겠지만 이를 개선할 방법은 여러가지가 있습니다. 특히 앞으로 살펴볼 CNN (Convolutional Neural Network) 에서는 훨씬 더 높은 정확도를 확인할 수 있을 것입니다. 